In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plot

In [ ]:
train_data = pd.read_csv('train.csv')
train_data = train_data.drop('label', axis=1)
train_data = train_data.as_matrix() / 255

In [ ]:
IMAGE_SIZE = 28
BATCH_SIZE = 32
NUM_ITERATIONS = 10000
HIDDEN_LAYERS_GEN = 128
LEARNING_RATE = 1e-4

In [ ]:
# Loading Batches
epochs_completed = 0
index_in_epoch = 0
num_examples = train_data.shape[0]


def next_batch(batch_size):
    global train_data
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        train_data = train_data[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    else:
        index_in_epoch = 0
        return next_batch(batch_size)
    end = index_in_epoch
    return train_data[start:end]

In [ ]:
def get_sample_z(size=(1, 100)):
    return np.random.normal(size=size)


def display_image(image_data):
    img = image_data.reshape([IMAGE_SIZE, IMAGE_SIZE])
    plot.axis('off')
    plot.imshow(img, cmap=matplotlib.cm.binary)
    plot.show()

In [ ]:
Z_in = tf.placeholder(tf.float32, shape=[None, 100])
image_in = tf.placeholder(tf.float32, shape=[None,
                                             IMAGE_SIZE * IMAGE_SIZE])


def generator(z):
    with tf.variable_scope('generator'):
        h = tf.layers.dense(z, HIDDEN_LAYERS_GEN)
        h = tf.minimum(h, 0.01)
        logits = tf.layers.dense(h, IMAGE_SIZE * IMAGE_SIZE)
        output = tf.nn.sigmoid(logits)
        return output


def discriminator(image, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        h = tf.layers.dense(image, HIDDEN_LAYERS_GEN)
        h = tf.minimum(h, 0.01)
        logits = tf.layers.dense(h, 1)
        return logits

In [ ]:
gen_sample = generator(Z_in)

discriminator_data = discriminator(image_in)
discriminator_model = discriminator(gen_sample, reuse=True)

In [ ]:
discriminator_loss = tf.reduce_mean(discriminator_data) - \
                     tf.reduce_sum(discriminator_model)

generator_loss = -tf.reduce_mean(discriminator_model)

In [ ]:
all_vars = tf.trainable_variables()
generator_vars = [var for var in all_vars if var.name.startswith('generator')]
discriminator_vars = [var for var in all_vars if var.name.startswith('discriminator')]

discriminator_optimize = tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE)\
    .minimize(discriminator_loss, var_list=discriminator_vars)
generator_optimize = tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE)\
    .minimize(generator_loss, var_list=generator_vars)

clip_D = [p.assign(tf.clip_by_value(p, -0.01, 0.01)) for p in discriminator_vars]

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(NUM_ITERATIONS):
    for a in range(5):
        image_batch = next_batch(BATCH_SIZE)
        _, disc_loss, _ = sess.run([discriminator_optimize, discriminator_loss, clip_D], 
                                   feed_dict={Z_in: get_sample_z([BATCH_SIZE, 100]),
                                              image_in: image_batch})
        
    _, gen_loss = sess.run([generator_optimize, generator_loss], 
                           feed_dict={Z_in: get_sample_z([BATCH_SIZE, 100])})
    
    if i % 100:
        print('Step {} => Discriminator: {} | Generator: {}'.format(i, disc_loss, gen_loss))
        
        if i % 1000 == 0:
            sample = sess.run(gen_sample, feed_dict={Z_in: get_sample_z()})
            display_image(sample)